In [ ]:
# Import necessary libraries
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import some common libraries
import numpy as np
import os, json, cv2, random

# Import Detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import  DatasetCatalog, MetadataCatalog
from pycocotools.coco import COCO
from detectron2.structures import BoxMode

In [ ]:
def get_kitti_mots_dicts(img_dir, ann_file):
    # Initialize COCO api for instance annotations
    coco = COCO(ann_file)
    
    dataset_dicts = []
    for idx, img_id in enumerate(coco.imgs):
        record = {}
        
        # Get image information
        img = coco.imgs[img_id]
        record["file_name"] = os.path.join(img_dir, img['file_name'])
        record["image_id"] = img_id
        record["height"] = img["height"]
        record["width"] = img["width"]
        
        # Read and process annotations
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)
        objs = []
        for ann in annotations:
            # Decode segmentation mask
            mask = coco.annToMask(ann)
            
            # Extract bounding box from mask
            x, y, w, h = cv2.boundingRect(mask)
            
            obj = {
                "bbox": [x, y, x+w, y+h],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": ann['segmentation'],
                "category_id": ann['category_id'],
                "iscrowd": 0
            }
            objs.append(obj)
        
        record["annotations"] = objs
        dataset_dicts.append(record)
    
    return dataset_dicts

def register_kitti_mots_datasets():
    # Path to KITTI-MOTS dataset
    kitti_mots_dir = '/path/to/KITTI-MOTS'
    for d in ["train", "val"]:
        DatasetCatalog.register("kitti_mots_" + d, lambda d=d: get_kitti_mots_dicts(os.path.join(kitti_mots_dir, d), os.path.join(kitti_mots_dir, f'instances_{d}.txt')))
        MetadataCatalog.get("kitti_mots_" + d).set(thing_classes=["car", "pedestrian"])

register_kitti_mots_datasets()


In [ ]:
# Setup configuration for Faster R-CNN
cfg_faster_rcnn = get_cfg()
cfg_faster_rcnn.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg_faster_rcnn.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg_faster_rcnn.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# Setup configuration for Mask R-CNN
cfg_mask_rcnn = get_cfg()
cfg_mask_rcnn.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_mask_rcnn.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg_mask_rcnn.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Assume you have a list of images in the KITTI-MOTS dataset
kitti_mots_images = ['/path/to/image1.jpg', '/path/to/image2.jpg']  # Update this with actual paths

# Run inference with Faster R-CNN
predictor_faster_rcnn = DefaultPredictor(cfg_faster_rcnn)
for image_path in kitti_mots_images:
    im = cv2.imread(image_path)
    outputs = predictor_faster_rcnn(im)
    # Here you can process the outputs, e.g., display or save them

# Run inference with Mask R-CNN
predictor_mask_rcnn = DefaultPredictor(cfg_mask_rcnn)
for image_path in kitti_mots_images:
    im = cv2.imread(image_path)
    outputs = predictor_mask_rcnn(im)
    # Here you can process the outputs, e.g., display or save them

In [ ]:
import numpy as np

np.random.randn(3, 4, 5).transpose(1,2,0)